In [ ]:
import numpy as np
import pandas as pd
import subprocess
import argparse
import datetime
import json
import requests
from requests.structures import CaseInsensitiveDict

#### Download MovieLens item and user data and prepare for reco output

In [ ]:
item_url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.item"

movie_dict = {}
user_dict = {}

genres = ["unknown", "action", "adventure", "animation", "children", "comedy", "crime", "documentary", "drama", 
          "fantasy", "noir", "horror", "musical", "mystery", "romance", "scifi", "thriller", "war", "western"] 
colnames=['id', 'title', 'release_date', 'video_release_date', 'imdb_url'] 
colnames.extend(genres)

item_df = pd.read_csv(item_url, sep='|', header=None, names=colnames, encoding='latin-1')

for idx, row in item_df.iterrows():
    movie_dict[row.id] = row.title

user_url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.user"

colnames=['id', 'age', 'gender', 'occupation', 'zip'] 
user_df = pd.read_csv(user_url, sep='|', header=None, names=colnames)

for idx, row in user_df.iterrows():
    user_dict[row.id] = row.occupation

print("Movies: " + str(len(movie_dict)) + " - Users: " + str(len(user_dict)))

**API-key** and **Domain-ID** which need to be copy/pasted from the Uptrendz platform:

In [ ]:
API_KEY = '<INSERT YOUR API KEY>'
DOMAIN_ID = '<INSERT YOUR DOMAIN ID>'

headers = {
    'x-api-key': API_KEY,
    'x-domain-id': DOMAIN_ID,
    'Content-Type': 'application/json'
}

test_user = "1"
count = 5
test_item = "9"

**Prerequisite**: Setup a popularity-based recommendation API with post-filtering on movie genre

https://github.com/lacic/ECIR2023Demo/blob/main/configure_reco_scenarios.md#popularity-with-post-filtering-on-movie-genre

In [ ]:
scenario = "popular-horror-movies"

response = requests.get('https://app.uptrendz.ai/sp/recomm?scenarioId=' + scenario + '&count=' + str(count) + '&userId=' + test_user, headers=headers)
response_dict = json.loads(response.text)

reco_movies = [movie_dict[int(x)] for x in response_dict["recommIds"]]
print("Most popular horror movies: \n" + ", ".join(reco_movies))

**Prerequisite**: Setup a recommendation API that is based on Collaborative Filtering

https://github.com/lacic/ECIR2023Demo/blob/main/configure_reco_scenarios.md#collaborative-filtering-using-rating-data

In [ ]:
scenario = "movies-based-on-my-ratings"

response = requests.get('https://app.uptrendz.ai/sp/recomm?scenarioId=' + scenario + '&count=' + str(count) + '&userId=' + test_user, headers=headers)
response_dict = json.loads(response.text)

reco_movies = [movie_dict[int(x)] for x in response_dict["recommIds"]]
print("Movies that people simiar to user " + test_user + " like: \n" + ", ".join(reco_movies))

**Prerequisite**: Setup a Content-based recommendation API to find similar movies

https://github.com/lacic/ECIR2023Demo/blob/main/configure_reco_scenarios.md#content-based-filtering-using-the-title-attribute

In [ ]:
scenario = "similar-movies"

response = requests.get('https://app.uptrendz.ai/sp/recomm?scenarioId=' + scenario + '&count=' + str(count) + '&userId=' + test_user + '&itemId=' + test_item, headers=headers)
response_dict = json.loads(response.text)

reco_movies = [movie_dict[int(x)] for x in response_dict["recommIds"]]
print("Movies that content-wise similar to " +  movie_dict[int(test_item)] + ": \n" + ", ".join(reco_movies))

**Prerequisite**: Setup a recommendation API that uses a hybrid approach to combine multiple algorithms

https://github.com/lacic/ECIR2023Demo/blob/main/configure_reco_scenarios.md#weighted-hybrid

In [ ]:
scenario = "hybrid-combination"

response = requests.get('https://app.uptrendz.ai/sp/recomm?scenarioId=' + scenario + '&count=' + str(count) + '&userId=' + test_user + '&itemId=' + test_item, headers=headers)
response_dict = json.loads(response.text)

reco_movies = [movie_dict[int(x)] for x in response_dict["recommIds"]]
print("Movies that are a result of a hybrid combination " +  movie_dict[int(test_item)] + ": \n" + ", ".join(reco_movies))

**Prerequisite**: Setup a user-based recommendation API


In [ ]:
scenario = "users-for-a-movie"

response = requests.get('https://app.uptrendz.ai/sp/recomm?scenarioId=' + scenario + '&count=' + str(count) + '&userId=' + test_user + '&itemId=' + test_item, headers=headers)
response_dict = json.loads(response.text)

reco_users = [x + "(" + user_dict[int(x)] + ")" for x in response_dict["recommIds"]]
print("Users that are relevant for movie " +  movie_dict[int(test_item)] + ": \n" + ", ".join(reco_users))